In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import pandas as pd
import csv
import warnings

In [2]:
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [3]:
# 크롬드라이버 호출
driver = webdriver.Chrome(executable_path = r'C:\Users\변정민\Downloads\chromedriver_win32\chromedriver.exe')
haksa_list = []
janghak_list = []
chaeyong_list = []

In [4]:
# 학사공지
driver.get('https://hufs.ac.kr/user/indexSub.action?codyMenuSeq=37080&siteId=hufs&menuType=T&uId=4&sortChar=AB&linkUrl=04_0202.html&mainFrame=right')
sleep(3)
#count = 1
#num = 2
page_num = 1

In [5]:
# 셀레니움 값이 있는지 없는지 체크하는 법. 새 공통 공지사항 올라오면 숫자가 변경되는 문제 발생, 공통이 아닌 학사공지는 한 페이지에 15개 고정
notice_num = 0
while True:
    try:
        notice_num += 1
        driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]/a').text
    except NoSuchElementException:
        notice_num = notice_num - 15  # 요소가 없을 경우 notice_num-1까지 존재, 학사공지는 15개이므로 추가로 14를 빼줌
        break
print(notice_num)

34


In [6]:
haksa_list = []

while len(haksa_list) <= 29:  # 리스트 개수는 공지사항 하나 당 제목과 URL 묶음이 한 개로 취급됨, 원하는 공지사항 개수 - 1로 적을 것. 공지 30개면 충분
    notice_list = {}
    try:
        if notice_num <= 48:
            withtag = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]').text
            sleep(0.5)
            
            title = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]/a').text
            sleep(0.5)

            link = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]/a')
            link.click()
            url = driver.current_url          
            sleep(1)

            driver.back()
            sleep(1)
            
        else:  # 다음 쪽으로 넘어감과 동시에 notice_num 초기화
            notice_num = 34
            
            page_num += 1
            page = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[2]/div[2]/ul/li[{page_num}]/a')
            page.click()
            sleep(1)
            
            withtag = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]').text
            sleep(0.5)
            
            title = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]/a').text
            sleep(0.5)

            link = driver.find_element_by_xpath(f'//*[@id="board-container"]/div[2]/form[1]/table/tbody/tr[{notice_num}]/td[2]/a')
            link.click()
            url = driver.current_url          
            sleep(1)

            driver.back()
            sleep(1)              
    except:
        pass
    
    # 태그 분류 작엄
    if withtag.startswith('[공통]') is True:
      tag = '[공통]'
    elif withtag.startswith('[글로벌]') is True:
      tag = '[글로벌]'
    elif withtag.startswith('[서울]') is True:
      tag = '[서울]'
    else:
      tag = '[태그없음]'

    notice_list['태그'] = tag
    notice_list['제목'] = title
    notice_list['총제목'] = withtag
    notice_list['URL'] = url
    haksa_list.append(notice_list)
    #count += 1
    notice_num += 1
    sleep(0.5)

In [7]:
print(haksa_list)

[{'태그': '[글로벌]', '제목': '2022학년도 제1학기 졸업시험 및 졸업논문 제출 일정 공고  ', '총제목': '[글로벌] 2022학년도 제1학기 졸업시험 및 졸업논문 제출 일정 공고  ', 'URL': 'https://hufs.ac.kr/user/indexSub.action?codyMenuSeq=37080&siteId=hufs&menuType=T&uId=4&sortChar=AB&linkUrl=04_0202.html&mainFrame=right&dum=dum&boardId=109336176&page=1&command=view&boardSeq=160618107'}, {'태그': '[공통]', '제목': '후기이중전공 신청 공고  ', '총제목': '[공통] 후기이중전공 신청 공고  ', 'URL': 'https://hufs.ac.kr/user/indexSub.action?codyMenuSeq=37080&siteId=hufs&menuType=T&uId=4&sortChar=AB&linkUrl=04_0202.html&mainFrame=right&dum=dum&boardId=109336176&page=1&command=view&boardSeq=160597872'}, {'태그': '[공통]', '제목': '(용인 삼계고) 교육봉사활동 멘토 모집 공고  ', '총제목': '[공통] (용인 삼계고) 교육봉사활동 멘토 모집 공고  ', 'URL': 'https://hufs.ac.kr/user/indexSub.action?codyMenuSeq=37080&siteId=hufs&menuType=T&uId=4&sortChar=AB&linkUrl=04_0202.html&mainFrame=right&dum=dum&boardId=109336176&page=1&command=view&boardSeq=160579346'}, {'태그': '[태그없음]', '제목': '2022 ＇성희롱 등 폭력예방교육＇ 안내  ', '총제목': '2022 ＇성희롱 등 폭력예방교육＇ 안내  ', 'U

In [8]:
df = pd.DataFrame(haksa_list)
df

,태그,제목,총제목,URL
0,[글로벌],2022학년도 제1학기 졸업시험 및 졸업논문 제출 일정 공고,[글로벌] 2022학년도 제1학기 졸업시험 및 졸업논문 제출 일정 공고,https://hufs.ac.kr/user/indexSub.action?codyMe...
1,[공통],후기이중전공 신청 공고,[공통] 후기이중전공 신청 공고,https://hufs.ac.kr/user/indexSub.action?codyMe...
2,[공통],(용인 삼계고) 교육봉사활동 멘토 모집 공고,[공통] (용인 삼계고) 교육봉사활동 멘토 모집 공고,https://hufs.ac.kr/user/indexSub.action?codyMe...
3,[태그없음],2022 ＇성희롱 등 폭력예방교육＇ 안내,2022 ＇성희롱 등 폭력예방교육＇ 안내,https://hufs.ac.kr/user/indexSub.action?codyMe...
4,[공통],2022-1학기 중간시험 공고,[공통] 2022-1학기 중간시험 공고,https://hufs.ac.kr/user/indexSub.action?codyMe...
5,[서울],"[서울] 교,사대생등 대학생 튜터링 장학사업 신청관련 추가 안내","[서울] 교,사대생등 대학생 튜터링 장학사업 신청관련 추가 안내",https://hufs.ac.kr/user/indexSub.action?codyMe...
6,[태그없음],2022학년도 제1학기 BRICs 융합전공 졸업논문 제출 일정 공고,2022학년도 제1학기 BRICs 융합전공 졸업논문 제출 일정 공고,https://hufs.ac.kr/user/indexSub.action?codyMe...
7,[공통],2022-여름계절학기 개설희망 교과목 수요조사 실시 공고,[공통] 2022-여름계절학기 개설희망 교과목 수요조사 실시 공고,https://hufs.ac.kr/user/indexSub.action?codyMe...
8,[글로벌],2022-1학기 폐강(2차) 공고(글로벌캠퍼스),[글로벌] 2022-1학기 폐강(2차) 공고(글로벌캠퍼스),https://hufs.ac.kr/user/indexSub.action?codyMe...
9,[공통],[공통]2022년 1학기 교육실습 진행관련 안내(교육실습 수강신청자 필독),[공통]2022년 1학기 교육실습 진행관련 안내(교육실습 수강신청자 필독),https://hufs.ac.kr/user/indexSub.action?codyMe...


In [ ]:
df.to_csv('학사.csv', index =False, encoding="UTF-8-sig")